<a href="https://colab.research.google.com/github/PaulSerin/Big-Data-Framework/blob/main/BDF_10_Spark_MLib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#00 - Configuration of Apache Spark on Collaboratory


###Installing Java, Spark, and Findspark


---


This code installs Apache Spark 2.2.1, Java 8, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.

In [1]:
import os

os.environ["SPARK_VERSION"] = "spark-3.5.3"
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget  http://apache.osuosl.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!echo $SPARK_VERSION-bin-hadoop3.tgz
!rm $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,734 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,512 kB]
Get:13 https://developer.download.nvidia.com/compute/cuda/

### Set Environment Variables
Set the locations where Spark and Java are installed.

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark/"
os.environ["DRIVE_DATA"] = "/content/gdrive/My Drive/Big Data Framework/data/"

!rm /content/spark
!ln -s /content/$SPARK_VERSION-bin-hadoop3 /content/spark
!export PATH=$PATH:$SPARK_HOME/bin:$SPARK_HOME/sbin
!echo $SPARK_HOME
!env |grep  "DRIVE_DATA"

rm: cannot remove '/content/spark': No such file or directory
/content/spark/
DRIVE_DATA=/content/gdrive/My Drive/Big Data Framework/data/


### Start a SparkSession
This will start a local Spark session.

In [3]:
!python -V

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

# Example: shows the PySpark version
print("PySpark version {0}".format(sc.version))

# Example: parallelise an array and show the 2 first elements
sc.parallelize([2, 3, 4, 5, 6]).cache().take(2)

Python 3.10.12
PySpark version 3.5.3


[2, 3]

In [4]:
from pyspark.sql import SparkSession
# We create a SparkSession object (or we retrieve it if it is already created)
spark = SparkSession \
.builder \
.appName("My application") \
.config("spark.some.config.option", "some-value") \
.master("local[4]") \
.getOrCreate()
# We get the SparkContext
sc = spark.sparkContext

In [5]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive



---


# 10 - Spark MLlib

Library of ML parallel algorithms for massive data

-   Machine learning classic algorithms: classification, regression, clustering, collaborative filtering
-   Other algorithms: feature extraction, transformation, dimensionality reduction, and selection
-   Tools to build, evaluate and adjust ML pipelines
-   Other tools: linear algebra, statistics, data processing, etc.


Two packages:

-   **spark.mllib:** Original RDD-based API
-   **spark.ml:** High-level API, based on DataFrames

Documentation and APIS:

- ML
    - Guide: http://spark.apache.org/docs/latest/ml-guide.html
    - API Python: https://spark.apache.org/docs/latest/api/python/reference/pyspark.ml.html
    - API Scala: https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.ml.package
- MLlib
    - Guide: http://spark.apache.org/docs/latest/mllib-guide.html
    - API Python: https://spark.apache.org/docs/latest/api/python/reference/pyspark.mllib.html
    - API Scala: https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.mllib.package



## Example

Use the [KMeans](http://spark.apache.org/docs/latest/mllib-clustering.html#k-means) clustering algorithm to group data from vectors spread over two clusters.


In [8]:
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.linalg import Vectors

#  Define an array of 4 sparse vectors, 3 elements each
sparseData = [
     Vectors.sparse(3, {1: 1.2}),
     Vectors.sparse(3, {1: 1.1}),
     Vectors.sparse(3, {0: 0.9, 2: 1.0}),
     Vectors.sparse(3, {0: 1.0, 2: 1.1})
 ]

for i in range(4):
    print(sparseData[i].toArray())

[0.  1.2 0. ]
[0.  1.1 0. ]
[0.9 0.  1. ]
[1.  0.  1.1]


In [7]:
# Turn the array into a DataFrame
dfSD = sc.parallelize([
  (1, sparseData[0]),
  (2, sparseData[1]),
  (3, sparseData[2]),
  (4, sparseData[3])
]).toDF(["row", "features"])

dfSD.show()

+---+-------------------+
|row|           features|
+---+-------------------+
|  1|      (3,[1],[1.2])|
|  2|      (3,[1],[1.1])|
|  3|(3,[0,2],[0.9,1.0])|
|  4|(3,[0,2],[1.0,1.1])|
+---+-------------------+



In [9]:
# Create a KMeans model without training, with 2 clusters
# For more information, see https://spark.apache.org/docs/latest/api/python/reference/pyspark.ml.html#module-pyspark.ml.clustering
kmeans = KMeans()\
    .setInitMode("k-means||")\
    .setFeaturesCol("features")\
    .setPredictionCol("prediction")\
    .setK(2)\
    .setSeed(1)

In [10]:
# Adjust the model to the previous DataFrame and show the cluster centres
kmModel = kmeans.fit(dfSD)
print("Clusters centres: {0}".format(
    kmModel.clusterCenters()))

Clusters centres: [array([0.  , 1.15, 0.  ]), array([0.95, 0.  , 1.05])]


In [11]:
# Verify that the model clusters the data from the previous array
kmModel.transform(dfSD).show()
# Calculate the cost as the addition of the squared distance between the input points
# and the centres of the corresponding clusters
print("Cost = {0}".format(
    kmModel.summary.trainingCost))

+---+-------------------+----------+
|row|           features|prediction|
+---+-------------------+----------+
|  1|      (3,[1],[1.2])|         0|
|  2|      (3,[1],[1.1])|         0|
|  3|(3,[0,2],[0.9,1.0])|         1|
|  4|(3,[0,2],[1.0,1.1])|         1|
+---+-------------------+----------+

Cost = 0.014999999999999236


In [12]:
# Test the model with other points
dfTest = sc.parallelize([
  (1, Vectors.sparse(3, {0: 0.9, 1:1.0, 2: 1.0})),
  (2, Vectors.sparse(3, {1: 1.5, 2: 0.3}))
]).toDF(["row", "features"])

kmModel.transform(dfTest).show(truncate=False)

# Calculate the cost as the addition of the squared distance between the input points
# and the centres of the corresponding clusters
print("Cost = {0}".format(
    kmModel.summary.trainingCost))

+---+-------------------------+----------+
|row|features                 |prediction|
+---+-------------------------+----------+
|1  |(3,[0,1,2],[0.9,1.0,1.0])|1         |
|2  |(3,[1,2],[1.5,0.3])      |0         |
+---+-------------------------+----------+

Cost = 0.014999999999999236


In [13]:
# Save the model in a local directory
kmModel.save("/tmp/kmModel")

In [14]:
# Reload the model
sameModel = KMeansModel.load("/tmp/kmModel")

sameModel.transform(dfTest).show(truncate=False)
# Calculate the cost as the addition of the squared distance between the input points
# and the centres of the corresponding clusters
# print("Cost = {0}".format(sameModel.summary.trainingCost))

+---+-------------------------+----------+
|row|features                 |prediction|
+---+-------------------------+----------+
|1  |(3,[0,1,2],[0.9,1.0,1.0])|1         |
|2  |(3,[1,2],[1.5,0.3])      |0         |
+---+-------------------------+----------+



In [15]:
!rm -rf /tmp/kmModel